In [ ]:
# This is a test of the initial files Jason gave me.
# Mostly this will be for developing a preprocessing script to get the forms into the proper format

In [ ]:
"""
Questions for Jason:
- Do we want to try and include goalie into the balancing?
- What fields do you want to keep for tracking purposes? (email?)
- When missing skill, what should we assume? (average? bad?)
- does Players Recent Team mean anything?
"""

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
pd.set_option('display.max_rows', None)

In [ ]:
division = "Boys Grades 3-4"
old_registration = "Spring2022-registrations.csv"
new_registration = "3-4boys-sample-22registration.csv"

In [ ]:
# Pull team and coach score from spring 2022
existing_players_raw = pd.read_csv(old_registration)
# Only keep teams if the players were in the relevant division.
# This is because a Spring 2nd-grader on Red shouldn't stay on Red, but
# we do want to know their skill score.
in_division = existing_players_raw["Division"].str.contains(division)
existing_players_raw.loc[~in_division, "Assigned Team"] = np.nan

column_map = {
    "Player rating - Effectiveness": "coach_skill",
    "Lastname": "last_name",
    "Firstname": "first_name",
    "Assigned Team": "team",
}
existing_players = existing_players_raw.rename(columns=column_map)[column_map.values()]

# Extract skill
existing_players["coach_skill"] = existing_players["coach_skill"].str.extract('(\d+)', expand=False).astype(float)

# Construct name for matching to current registration.
full_name = existing_players["first_name"] + existing_players["last_name"]
normalized_name = full_name.str.lower().str.replace('[^a-zA-Z]', '', regex=True)
existing_players["name_key"] = normalized_name
existing_players = (
    existing_players
    .drop(columns=["first_name", "last_name"])
    # there seemed to be duplicate rows (with missing skill scores) so 
    # let's take the highest score for each name. Note this assumes
    # that kids have unique name keys
    .sort_values(by="coach_skill")
    .drop_duplicates("name_key")
)

print(f"Loaded {len(existing_players)} existing players for skill/team lookup.")
print(f"{in_division.sum()} were found in division: {division}")
existing_players.head(15)

In [ ]:
# Prepare location finding
import logging
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="cyslf")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1/2)

def lookup_location(address):
    # If an address is poorly formed, geopy gives sad-looking warnings,
    # so let's temporarily disable this.
    logging.getLogger("geopy").setLevel(logging.ERROR)
    location = geocode(address)
    logging.getLogger("geopy").setLevel(logging.WARNING)
    if location:
        return location.latitude, location.longitude
    else:
        print(f"Failed to find address: {address}")
        return np.nan, np.nan

In [ ]:
# Load current registrations

registrations_raw = pd.read_csv(new_registration)

# Join the two school columns
has_other_school = ~pd.isnull(registrations_raw["School Name other:"])
registrations_raw.loc[has_other_school, "School Name"] = registrations_raw[has_other_school]["School Name other:"]

# Look up player latitude / longitude
registrations_raw["Postal Code"] = registrations_raw["Postal Code"].astype(str)
registrations_raw["Address"] = registrations_raw[["Street", "City", "Region", "Postal Code"]].agg(", ".join, axis=1)
registrations_raw["Location"] = registrations_raw['Address'].progress_apply(lookup_location)
registrations_raw["latitude"] = registrations_raw["Location"].apply(lambda x: x[0])
registrations_raw["longitude"] = registrations_raw["Location"].apply(lambda x: x[1])

In [ ]:
column_map = {
    "Player Last Name": "last_name",
    "Player First Name": "first_name",
    "Current Grade": "grade",
    "Parental assessment of player ability/athleticism:": "parent_skill",
    "School Name": "school",
    "Special Requests": "comment",
    "longitude": "longitude",
    "latitude": "latitude",
    "Teammate Request": "requested_teammate",
}
registrations = registrations_raw.rename(columns=column_map)[column_map.values()]

# Extract grade
registrations["grade"] = registrations["grade"].str.extract('(\d+)', expand=False).astype(int)

# Extract skill
registrations["parent_skill"] = registrations["parent_skill"].str.extract('(\d+)', expand=False).astype(float)
registrations["parent_skill"] = 11 - registrations["parent_skill"] # These have the opposite order of coach skill, so invert.

# Construct name for matching to current registration.
full_name = registrations["first_name"] + registrations["last_name"]
normalized_name = full_name.str.lower().str.replace('[^a-zA-Z]', '', regex=True)
registrations["name_key"] = normalized_name

print(f"Found {len(registrations)} registrations in division: {division}")
registrations.head(5)

In [ ]:
existing_player_match = registrations["name_key"].isin(existing_players["name_key"])
print(f"{existing_player_match.sum()} out of {len(registrations)} players were matched to existing player data.")
print("The following players were NOT matched:")
registrations[~existing_player_match].head(20)

In [ ]:
ordered_columns = [
    "id",
    "last_name",
    "first_name",
    "grade",
    "team",
    "coach_skill",
    "parent_skill",
    "longitude",
    "latitude",
    "preferred_days",
    "unavailable_days",
    "frozen",
    "school",
    "comment",
]
players = registrations.merge(existing_players, how="left", on="name_key")
players["id"] = players.index.values

# Freeze players to a team if they already have one
players["frozen"] = True
missing_team = pd.isnull(players.team)
players.loc[missing_team, "frozen"] = False

for col in ordered_columns:
    if col not in players.columns:
        players[col] = np.nan

players = players[ordered_columns]

print(f"Total # of players: {len(players)}")
players.head()

In [ ]:
print("Verification checks:")

full_name = players["first_name"] + players["last_name"]
normalized_name = full_name.str.lower().str.replace('[^a-zA-Z]', '', regex=True)
name_counts = normalized_name.value_counts()
duplicate_names = name_counts[name_counts > 1].index.values
has_duplicate = normalized_name.isin(duplicate_names)
if has_duplicate.sum() > 0:
    print(f"{has_duplicate.sum()} rows have names that match other rows, so please check if they're duplicates:")
    print(players[has_duplicate].sort_values(by="last_name")[["id", "first_name", "last_name"]])

missing_last_name = pd.isnull(players.last_name)
if missing_last_name.sum() > 0:
    print(f"{missing_last_name.sum()} players are missing a last name:")
    print(players[missing_last_name][["id", "first_name", "last_name"]])
    print()

missing_first_name = pd.isnull(players.first_name)
if missing_first_name.sum() > 0:
    print(f"{missing_first_name.sum()} players are missing a first name:")
    print(players[missing_first_name][["id", "first_name", "last_name"]])
    print()

missing_grade = pd.isnull(players.grade)
if missing_grade.sum() > 0:
    print(f"{missing_grade.sum()} players are missing a grade:")
    print(players[missing_grade][["id", "first_name", "last_name", "grade"]])
    print()

missing_skill = pd.isnull(players.coach_skill) & pd.isnull(players.parent_skill)
if missing_skill.sum() > 0:
    print(f"{missing_skill.sum()} players don't have a coach or parent skill:")
    print(players[missing_skill][["id", "first_name", "last_name", "coach_skill", "parent_skill"]])
    print()
    
    # For testing let's give them the worst score
    # I should ask Jason about this though
    players.loc[missing_skill, "parent_skill"] = 10
    
print()
print("Please manually fix any listed problems before the next step^")

In [ ]:
player_outfile = "fall-2022-players.csv"
print(f"Saving to {player_outfile}")
players.to_csv(player_outfile, index=False)

# TEAM

In [ ]:
team_names = players.team.dropna().unique()
locations = [
    (42.38980588, -71.1330116),
    (42.37292539, -71.083611),
    (42.37685248, -71.12084011),
]
team_records = []
for i, team_name in enumerate(team_names):
    location = locations[i % len(locations)]
    team_records.append({
        "name": team_name,
        "practice_day": "M",
        "latitude": location[0],
        "longitude": location[1],
    })
teams = pd.DataFrame.from_records(team_records)
teams

In [ ]:
team_outfile = "fall-2022-teams.csv"
print(f"Saving to {team_outfile}")
teams.to_csv(team_outfile, index=False)

# Assign teams

In [ ]:
from tqdm import tqdm
import configparser
from cyslf.models import League
from cyslf.optimize import find_best_moves

input_stem = "fall-2022"
output_stem = "fall-2022-out"
config_file = "weights.cfg"

league = League.from_csvs(
    f"{input_stem}-players.csv", f"{input_stem}-teams.csv"
)

weights = None
if config_file is not None:
    config = configparser.ConfigParser()
    config.read(config_file)
    weights = {}
    for key in config["weights"].keys():
        weights[key] = float(config["weights"][key])

for i in tqdm(range(len(league.available_players))):
    player = league.get_next_player()
    best_moves = find_best_moves(player, league, depth=2, weights=weights)
    league.apply_moves(best_moves)
league.details()

# TODO: add override check
league.to_csvs(f"{output_stem}-players.csv", f"{output_stem}-teams.csv")